In [165]:
import pyspark

In [166]:
import pandas as pd

# Replace 'your_file.csv' with the actual file path
file_path = 'Chicago_Crimes_2012_to_2017.csv'

# Specify the delimiter used in your CSV file (e.g., ';')
df = pd.read_csv(file_path, sep=';')

In [167]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Dataframe').getOrCreate()

In [168]:
## read the dataset
df_pyspark=spark.read.option('header','true').csv('Chicago_Crimes_2012_to_2017.csv',inferSchema=True)

In [169]:
df_pyspark=df_pyspark.drop('_c0','ID','Block','IUCR','Description','Location Description','X Coordinate','Y Coordinate','Latitude','Longitude','Location')

In [170]:
from pyspark.sql.functions import col

# Assume df_pyspark is your DataFrame
# df_pyspark = ...

# Iterate over the columns and count null values
for column in df_pyspark.columns:
    null_count = df_pyspark.filter(col(column).isNull()).count()
    print(f"Number of null values in '{column}': {null_count}")


Number of null values in 'Case Number': 1


Number of null values in 'Date': 0


Number of null values in 'Primary Type': 0


Number of null values in 'Arrest': 0


Number of null values in 'Domestic': 0


...

Number of null values in 'Ward': 14


Number of null values in 'Community Area': 40


Number of null values in 'FBI Code': 0


Number of null values in 'Year': 0


Number of null values in 'Updated On': 0


In [171]:
df_pyspark.columns

['Case Number',
 'Date',
 'Primary Type',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'Year',
 'Updated On']

In [172]:
from pyspark.ml.feature import Imputer
from pyspark.sql.functions import when, col

# Define the columns to impute
input_cols = ['Ward', 'Community Area', 'District']

# Create an Imputer object
imputer = Imputer(
    inputCols=input_cols,
    outputCols=["{}_imputed".format(c) for c in input_cols]
).setStrategy("mean")

# Fit and transform the DataFrame
df_imputed = imputer.fit(df_pyspark).transform(df_pyspark)

# Overwrite 'Ward', 'Community Area', and 'District' columns with imputed values
df_imputed_overwritten = df_imputed.withColumn(
    'Ward', 
    when(col('Ward').isNull(), col('Ward_imputed')).otherwise(col('Ward'))
).withColumn(
    'Community Area', 
    when(col('Community Area').isNull(), col('Community Area_imputed')).otherwise(col('Community Area'))
).withColumn(
    'District', 
    when(col('District').isNull(), col('District_imputed')).otherwise(col('District'))
)

# Drop the intermediate imputed columns
df_imputed_overwritten = df_imputed_overwritten.drop('Ward_imputed', 'Community Area_imputed', 'District_imputed')

# Update df_pyspark with the overwritten DataFrame
df_pyspark = df_imputed_overwritten


In [173]:
from pyspark.sql.functions import col

# Drop rows with null values in the 'Case Number' column
df_pyspark = df_pyspark.na.drop(subset=['Case Number'])


In [174]:
from pyspark.sql.functions import col

# Assume df_pyspark is your DataFrame
# df_pyspark = ...

# Iterate over the columns and count null values
for column in df_pyspark.columns:
    null_count = df_pyspark.filter(col(column).isNull()).count()
    print(f"Number of null values in '{column}': {null_count}")


Number of null values in 'Case Number': 0


Number of null values in 'Date': 0


Number of null values in 'Primary Type': 0


Number of null values in 'Arrest': 0


Number of null values in 'Domestic': 0


...

Number of null values in 'Ward': 0


Number of null values in 'Community Area': 0


Number of null values in 'FBI Code': 0


Number of null values in 'Year': 0


Number of null values in 'Updated On': 0
